# transfer-learning
model trained by the transfer-value from the pretrained models: Inception V3, Inception V4, Inception-Resnet-V2, Resnet 152.

In [1]:
import tensorflow as tf
from tensorflow.python.platform import tf_logging as logging

import os
import time
from tensorflow.contrib import slim

from datasets import dataset_utils

In [2]:
import tensorflow as tf
from tensorflow.contrib import slim
from transfer_value_maker_loader import get_split_tv, load_batch_tv, read_shape_file

In [41]:
MODEL = 'inception_resnet_v2'
# MODELs = ['inception_v3', 
#           'inception_v4', 
#           'inception_resnet_v2',
#           'resnet_v2_152']

dataset_dir = '../transfer-value-tf/' + MODEL

num_classes = 10

split_name = 'train'
# split_names = ['train', 'validation', 'test']

file_pattern_train = 'train*.tfrecord'
file_pattern_val = 'validation*.tfrecord'
file_pattern = split_name + '*.tfrecord'

checkpoints_dir = '../checkpoints/' + MODEL

log_dir = '../log/' + MODEL
if not tf.gfile.Exists(log_dir):
    tf.gfile.MakeDirs(log_dir)

In [40]:
num_epochs = 20

batch_size = 32

dropout_rate = 0.4

fc_units = 512

num_epochs_before_decay = 2

initial_learning_rate = 0.001

learning_rate_decay_factor = 0.7

In [15]:
shape = read_shape_file(MODEL, dataset_dir)
shape

[1, 1, 2048]

In [31]:
def model(inputs, fc_units, num_classes, dropout_rate, is_training=True):
    net = tf.squeeze(inputs, [1, 2], name='squeeze')
    net = tf.layers.dense(net, fc_units, activation=tf.nn.relu, name='fc')
    net = tf.layers.dropout(net, rate=dropout_rate, seed=42, training=is_training, name='dropout')
    logits = tf.layers.dense(net, units=num_classes, name='logits')
    probabilities = tf.nn.softmax(logits, name='softmax_output')
    return logits, probabilities

In [31]:
def model_pre(inputs, fc_units, num_classes, dropout_rate, is_training=True):
    net = tf.layers.dense(inputs, fc_units, activation=tf.nn.relu, name='fc')
    net = tf.layers.dropout(net, rate=dropout_rate, seed=42, training=is_training, name='dropout')
    logits = tf.layers.dense(net, units=num_classes, name='logits')
    probabilities = tf.nn.softmax(logits, name='softmax_output')
    return logits, probabilities

In [22]:
def model_fc2(inputs, num_classes, dropout_rate, is_training=True):
    net = tf.layers.dense(inputs, 1024, activation=tf.nn.relu, name='fc1')
    net = tf.layers.dropout(net, rate=dropout_rate, seed=42, training=is_training, name='dropout1')
    net = tf.layers.dense(net, 512, activation=tf.nn.relu, name='fc2')
    net = tf.layers.dropout(net, rate=dropout_rate, seed=42, training=is_training, name='dropout2')
    logits = tf.layers.dense(net, units=num_classes, name='logits')
    probabilities = tf.nn.softmax(logits, name='softmax_output')
    return logits, probabilities

In [43]:
def model_sp(inputs, num_classes, dropout_rate, is_training=True):
    net = tf.layers.dropout(inputs, rate=dropout_rate, seed=42, training=is_training, name='dropout')
    logits = tf.layers.dense(net, units=num_classes, name='logits')
    probabilities = tf.nn.softmax(logits, name='softmax_output')
    return logits, probabilities

In [42]:
num_samples = 0
file_pattern_for_counting = 'train'
tfrecords_to_count = [os.path.join(dataset_dir, file) for file in os.listdir(dataset_dir) if file.startswith(file_pattern_for_counting)]
if len(tfrecords_to_count) == 0:
    raise ValueError('There is no dataset.')
for tfrecord_file in tfrecords_to_count:
    for record in tf.python_io.tf_record_iterator(tfrecord_file):
        num_samples += 1
num_samples

17940

In [4]:
import numpy as np
import pandas as pd

In [5]:
def load_tv(MODEL, split_name, dataset_dir): 
    with tf.Graph().as_default() as graph:
        tf.logging.set_verbosity(tf.logging.INFO)
        
        file_pattern = split_name + '*.tfrecord'
        dataset = get_split_tv(split_name, dataset_dir, file_pattern)
        
        shape = read_shape_file(MODEL, dataset_dir)
        
        arr_, label_, filename_ = load_batch_tv(dataset, 1, MODEL, shape, is_training=True)

        sv = tf.train.Supervisor(logdir='../log/tmp')

        with sv.managed_session() as sess:
            list_ = []
            for i in range(dataset.num_samples):
                arrs, labels, filenames = sess.run([arr_, label_, filename_])
                arr, label, filename = list(np.squeeze(arrs[0])), [labels[0]], [filenames[0].decode()]
                row = filename + label + arr
                list_.append(row)
            df = pd.DataFrame(list_) 
            return df

In [6]:
def load_batches(dataset, batch_size):
    num_samples = len(dataset)
    num_batches = num_samples // batch_size + 1
    for step in range(num_batches):
        start = step * batch_size
        end = min((step + 1) * batch_size, num_samples)
        labels = dataset.iloc[start:end, 1]
        inputs = dataset.iloc[start:end, 2:]
        yield inputs, labels

In [7]:
def load_val(dataset):
    labels = dataset.iloc[:, 1]
    inputs = dataset.iloc[:, 2:]
    return inputs, labels

In [8]:
def clean_dir(directory):
    import shutil
    if os.path.exists(directory):
        shutil.rmtree(directory) 

In [153]:
clean_dir(log_dir)

In [55]:
def train(MODEL, fc_units, num_epochs, batch_size, dropout_rate_train=0.4, initial_learning_rate=0.001, new_training=True):
    dataset_dir = '../transfer-value-tf/' + MODEL
    num_classes = 10
    print(dataset_dir)

    dataset_train = load_tv(MODEL, 'train', dataset_dir)
    dataset_val = load_tv(MODEL, 'validation', dataset_dir)
    
    log_dir = '../log/' + MODEL
    if new_training:
        clean_dir(log_dir)
    if not tf.gfile.Exists(log_dir):
        tf.gfile.MakeDirs(log_dir)
        
    with tf.Graph().as_default() as graph:
        tf.logging.set_verbosity(tf.logging.INFO)

        shape = read_shape_file(MODEL, dataset_dir)

        inputs = tf.placeholder(tf.float32, [None, shape[2]], name='input')
        dropout_rate = tf.placeholder(tf.float32, name='dropout_rate')
        labels = tf.placeholder(tf.int32, name='labels')

        logits, probs = model_pre(inputs, fc_units, num_classes, dropout_rate, is_training=True)

        labels_oh = tf.contrib.layers.one_hot_encoding(labels, num_classes, scope='labels_oh')
        # labels_oh = slim.one_hot_encoding(labels, num_classes, scope='labels_oh')


        loss = tf.losses.softmax_cross_entropy(onehot_labels=labels_oh, logits=logits)
#         total_loss = tf.losses.get_total_loss()

        global_step = tf.train.get_or_create_global_step()
#         global_step_op = tf.assign(global_step, global_step + 1)

        learning_rate = tf.train.exponential_decay(learning_rate=initial_learning_rate, 
                                                   global_step=global_step, 
                                                   decay_steps=17940*2, 
                                                   decay_rate=0.8, 
                                                   staircase=True)

        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
#         train_op = slim.learning.create_train_op(total_loss, optimizer)

        predictions = tf.argmax(probs, 1)
        
        # accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, labels)
        # metrics_op = tf.group(accuracy_update, probabilities)

        _, accuracy = tf.metrics.accuracy(labels=labels, predictions=predictions)

        tf.summary.scalar('losses/Total_Loss', loss)
        tf.summary.scalar('accuracy', accuracy)
        tf.summary.scalar('learning_rate', learning_rate)
        my_summary_op = tf.summary.merge_all()
        
#         checkpoint_file = tf.train.latest_checkpoint(log_dir)
#         saver = tf.train.Saver()       
#         def restore_fn(sess):
#             return saver.restore(sess, checkpoint_file)

        sv = tf.train.Supervisor(logdir=log_dir, summary_op=None)

        with sv.managed_session() as sess:
#         with tf.Session() as sess:
            # Initializing the variables
#             sess.run(tf.global_variables_initializer())
            # saver.tf.train.Saver()
#             step = 0
            for i in range(num_epochs):
                inputs_v, labels_v = load_val(dataset_val)
#                 total_losses = []
#                 total_accuracies = []
                for inputs_t, labels_t in load_batches(dataset_train, batch_size=32):
#                     step = sess.run(global_step_op)
                    _, loss_t, acc_t = sess.run([optimizer, loss, accuracy], feed_dict={inputs: inputs_t, labels: labels_t, dropout_rate: dropout_rate_train})
#                     summaries = sess.run(my_summary_op)
#                     sv.summary_computed(sess, summaries)
#                     if step % 10 == 0:
#                         summaries = sess.run(my_summary_op)
#                         sv.summary_computed(sess, summaries)
# #                     total_losses.append(loss_t)
#                     total_accuracies.append(acc_t)  
                    # saver.save(sess, log_dir + '/model', global_step=step)
#                 total_loss = np.mean(total_losses)
#                 total_accuracy = np.mean(total_accuracies)
                logging.info('Epoch {}/{}'.format(i, num_epochs))
                loss_v, acc_v = sess.run([loss, accuracy], feed_dict={inputs: inputs_v, labels: labels_v, dropout_rate: 0})
                logging.info('Training Loss: {}， Training Accuracy: {}.'.format(loss_t, acc_t))
                logging.info('Validation Loss: {}, Validation Accuracy: {}.'.format(loss_v, acc_v))
            logging.info('Training finished! Saving model to disk.')
            sv.saver.save(sess, sv.save_path, global_step=sv.global_step)

In [24]:
train(MODEL='inception_resnet_v2', 
      fc_units=512, 
      num_epochs=20, 
      batch_size=64, 
      dropout_rate_train=0.4, 
      initial_learning_rate=0.001, 
      new_training=True)

../transfer-value-tf/inception_resnet_v2
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Recording summary at step None.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Recording summary at step None.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Saving checkpoint to path ../log/inception_resnet_v2\model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Epoch 0/20
INFO:tensorflow:Training Loss: 0.9082881212234497， Training Accuracy: 0.4591697156429291.
INFO:tensorflow:Validation Loss: 0.6734882593154907, Validation Accuracy: 0.6357027888298035.
INFO:tensorflow:Epoch 1/20
INFO:tensorflow:Training Loss: 0.6417461037635803， Training Accuracy: 0.6772149205207825.
INFO:tensorflow:Validation Loss: 0.44153261184692383, Validation Accuracy: 0.7268774509429932.
INFO:tensorflow:Epoch 2/20
INFO:tensorflow:Training Loss: 0.4880469441413

In [32]:
def train(MODEL, fc_units, num_epochs, batch_size, dropout_rate_train=0.4, initial_learning_rate=0.001, new_training=True):
    dataset_dir = '../transfer-value-tf/' + MODEL
    num_classes = 10
    print(dataset_dir)

    dataset_train = load_tv(MODEL, 'train', dataset_dir)
    dataset_val = load_tv(MODEL, 'validation', dataset_dir)
    
    log_dir = '../log/' + MODEL
    if new_training:
        clean_dir(log_dir)
    if not tf.gfile.Exists(log_dir):
        tf.gfile.MakeDirs(log_dir)
        
    with tf.Graph().as_default() as graph:
        tf.logging.set_verbosity(tf.logging.INFO)

        shape = read_shape_file(MODEL, dataset_dir)

        inputs = tf.placeholder(tf.float32, [None, shape[2]], name='input')
        dropout_rate = tf.placeholder(tf.float32, name='dropout_rate')
        labels = tf.placeholder(tf.int32, name='labels')

        logits, probs = model_pre(inputs, fc_units, num_classes, dropout_rate, is_training=True)

        labels_oh = tf.contrib.layers.one_hot_encoding(labels, num_classes, scope='labels_oh')

        loss = tf.losses.softmax_cross_entropy(onehot_labels=labels_oh, logits=logits)
        total_loss = tf.losses.get_total_loss()

        global_step = tf.train.get_or_create_global_step()

        learning_rate = tf.train.exponential_decay(learning_rate=initial_learning_rate, 
                                                   global_step=global_step, 
                                                   decay_steps=17940*2, 
                                                   decay_rate=0.8, 
                                                   staircase=True)

        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = slim.learning.create_train_op(total_loss, optimizer)

        predictions = tf.argmax(probs, 1)
        
        _, accuracy = tf.metrics.accuracy(labels=labels, predictions=predictions)

        tf.summary.scalar('losses/Total_Loss', total_loss)
        tf.summary.scalar('accuracy', accuracy)
        tf.summary.scalar('learning_rate', learning_rate)
        my_summary_op = tf.summary.merge_all()
        
        sv = tf.train.Supervisor(logdir=log_dir, summary_op=None)

        with sv.managed_session() as sess:
            for i in range(num_epochs):
                inputs_v, labels_v = load_val(dataset_val)
                acc = []
                for inputs_t, labels_t in load_batches(dataset_train, batch_size=32):
                    loss_t, acc_t, step = sess.run([train_op, accuracy, sv.global_step], feed_dict={inputs: inputs_t, labels: labels_t, dropout_rate: dropout_rate_train})
                    acc.append(acc_t)
                    if step % 10 == 0:
                        summaries = sess.run(my_summary_op, feed_dict={inputs: inputs_t, labels: labels_t, dropout_rate: dropout_rate_train})
                        sv.summary_computed(sess, summaries)
                acc_mean = np.mean(np.array(acc))    
                logging.info('Epoch {}/{}'.format(i, num_epochs))
                loss_v, acc_v = sess.run([loss, accuracy], feed_dict={inputs: inputs_v, labels: labels_v, dropout_rate: 0})
                logging.info('Training Loss: {}， Training Accuracy: {}.'.format(loss_t, acc_mean))
                logging.info('Validation Loss: {}, Validation Accuracy: {}.'.format(loss_v, acc_v))
            logging.info('Training finished! Saving model to disk.')
            sv.saver.save(sess, sv.save_path, global_step=sv.global_step)

In [34]:
train(MODEL='inception_v3', 
      fc_units=128, 
      num_epochs=20, 
      batch_size=32, 
      dropout_rate_train=0.4, 
      initial_learning_rate=0.001, 
      new_training=True)

../transfer-value-tf/inception_v3
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Recording summary at step None.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Recording summary at step None.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path ../log/inception_v3\model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Epoch 0/20
INFO:tensorflow:Training Loss: 0.9539110064506531， Training Accuracy: 0.397233784198761.
INFO:tensorflow:Validation Loss: 0.6465864777565002, Validation Accuracy: 0.5972084403038025.
INFO:tensorflow:Epoch 1/20
INFO:tensorflow:Training Loss: 0.7335414290428162， Training Accuracy: 0.6351739168167114.
INFO:tensorflow:Validation Loss: 0.38152429461479187, Validation Accuracy: 0.6898125410079956.
INFO:tensorflow:Epoch 2/20
INFO:tensorflow:Training Loss: 0.6140457987785339， Training Accuracy: 0.7092317938804626.
INFO:t

In [9]:
from tqdm import tqdm

In [37]:
def evaluation(MODEL, fc_units):
    dataset_dir = '../transfer-value-tf/' + MODEL
    num_classes = 10

#     dataset = load_tv(MODEL, 'test', dataset_dir)
#     print('dataset is loaded.')
    
    log_dir = '../log/' + MODEL
    if not tf.gfile.Exists(log_dir):
        tf.gfile.MakeDirs(log_dir)
        
    with tf.Graph().as_default() as graph:
        tf.logging.set_verbosity(tf.logging.INFO)

        shape = read_shape_file(MODEL, dataset_dir)

        inputs = tf.placeholder(tf.float32, [None, shape[2]], name='input')
        dropout_rate = tf.placeholder(tf.float32, name='dropout_rate')
        labels = tf.placeholder(tf.int32, name='labels')

        _, probs = model_pre(inputs, fc_units, num_classes, dropout_rate, is_training=True)
        
        checkpoint_file = tf.train.latest_checkpoint(log_dir)
        # create saver function to restore variables from a checkpoint file
        saver = tf.train.Saver()
        def restore_fn(sess):
            return saver.restore(sess, checkpoint_file)
        clean_dir('../log/test')
        sv = tf.train.Supervisor(logdir='../log/test', summary_op=None, saver=None, init_fn=restore_fn)

        with sv.managed_session() as sess:
            predictions = [['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']]
            for i in tqdm(range(len(dataset))):
                array = [list(dataset.iloc[i, 2:])]
                prob = sess.run(probs, feed_dict={inputs: array, dropout_rate: 0})
                prob = list(prob[0])
                img_name = [dataset.iloc[i, 0]]
                prob = img_name + prob
                predictions.append(prob)
            pd.DataFrame(predictions).to_csv('../submission'+MODEL+'.csv', index=False, header=False)

In [38]:
evaluation(MODEL='inception_v3', fc_units=128)

INFO:tensorflow:Restoring parameters from ../log/inception_v3\model.ckpt-11220
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.


100%|██████████| 79726/79726 [05:49<00:00, 118.60it/s]


In [49]:
def train_sp(MODEL, num_epochs, batch_size, dropout_rate_train=0.4, initial_learning_rate=0.001, new_training=True):
    dataset_dir = '../transfer-value-tf/' + MODEL
    num_classes = 10
    print(dataset_dir)

    dataset_train = load_tv(MODEL, 'train', dataset_dir)
    dataset_val = load_tv(MODEL, 'validation', dataset_dir)
    
    log_dir = '../log/' + MODEL
    if new_training:
        clean_dir(log_dir)
    if not tf.gfile.Exists(log_dir):
        tf.gfile.MakeDirs(log_dir)
        
    with tf.Graph().as_default() as graph:
        tf.logging.set_verbosity(tf.logging.INFO)

        shape = read_shape_file(MODEL, dataset_dir)

        inputs = tf.placeholder(tf.float32, [None, shape[2]], name='input')
        dropout_rate = tf.placeholder(tf.float32, name='dropout_rate')
        labels = tf.placeholder(tf.int32, name='labels')

        logits, probs = model_sp(inputs, num_classes, dropout_rate, is_training=True)

        labels_oh = tf.contrib.layers.one_hot_encoding(labels, num_classes, scope='labels_oh')

        loss = tf.losses.softmax_cross_entropy(onehot_labels=labels_oh, logits=logits)
        total_loss = tf.losses.get_total_loss()

        global_step = tf.train.get_or_create_global_step()

#         learning_rate = tf.train.exponential_decay(learning_rate=initial_learning_rate, 
#                                                    global_step=global_step, 
#                                                    decay_steps=17940*2, 
#                                                    decay_rate=0.8, 
#                                                    staircase=True)

        learning_rate = initial_learning_rate

        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = slim.learning.create_train_op(total_loss, optimizer)

        predictions = tf.argmax(probs, 1)
        
        _, accuracy = tf.metrics.accuracy(labels=labels, predictions=predictions)

        tf.summary.scalar('losses/Total_Loss', total_loss)
        tf.summary.scalar('accuracy', accuracy)
        tf.summary.scalar('learning_rate', learning_rate)
        my_summary_op = tf.summary.merge_all()
        
        sv = tf.train.Supervisor(logdir=log_dir, summary_op=None)

        with sv.managed_session() as sess:
            for i in range(num_epochs):
                inputs_v, labels_v = load_val(dataset_val)
                acc = []
                for inputs_t, labels_t in load_batches(dataset_train, batch_size=32):
                    loss_t, acc_t, step = sess.run([train_op, accuracy, sv.global_step], feed_dict={inputs: inputs_t, labels: labels_t, dropout_rate: dropout_rate_train})
                    acc.append(acc_t)
                    if step % 10 == 0:
                        summaries = sess.run(my_summary_op, feed_dict={inputs: inputs_t, labels: labels_t, dropout_rate: dropout_rate_train})
                        sv.summary_computed(sess, summaries)
                acc_mean = np.mean(np.array(acc))
                logging.info('Epoch {}/{}'.format(i, num_epochs))
                loss_v, acc_v = sess.run([loss, accuracy], feed_dict={inputs: inputs_v, labels: labels_v, dropout_rate: 0})
                logging.info('Training Loss: {}， Training Accuracy: {}.'.format(loss_t, acc_mean))
                logging.info('Validation Loss: {}, Validation Accuracy: {}.'.format(loss_v, acc_v))
            logging.info('Training finished! Saving model to disk.')
            sv.saver.save(sess, sv.save_path, global_step=sv.global_step)

In [57]:
train_sp(MODEL='inception_resnet_v2', 
         num_epochs=500, 
         batch_size=32, 
         dropout_rate_train=0.3, 
         initial_learning_rate=0.001, 
         new_training=True)

../transfer-value-tf/inception_resnet_v2
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Recording summary at step None.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Recording summary at step None.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path ../log/inception_resnet_v2\model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Epoch 0/500
INFO:tensorflow:Training Loss: 1.153900384902954， Training Accuracy: 0.3986290991306305.
INFO:tensorflow:Validation Loss: 1.026543140411377, Validation Accuracy: 0.5610753297805786.
INFO:tensorflow:Epoch 1/500
INFO:tensorflow:Training Loss: 0.9916263818740845， Training Accuracy: 0.597919762134552.
INFO:tensorflow:Validation Loss: 0.7873894572257996, Validation Accuracy: 0.6438305377960205.
INFO:tensorflow:Epoch 2/500
INFO:tensorflow:Training Loss: 0.6557033658027649， Training Accuracy: 0.6623143553

In [53]:
dataset = load_tv('inception_resnet_v2', 'test', '../transfer-value-tf/'+'inception_resnet_v2')

INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Recording summary at step None.
INFO:tensorflow:Recording summary at step None.


In [55]:
def evaluation_sp(MODEL):
    dataset_dir = '../transfer-value-tf/' + MODEL
    num_classes = 10

#     dataset = load_tv(MODEL, 'test', dataset_dir)
#     print('dataset is loaded.')
    
    log_dir = '../log/' + MODEL
    if not tf.gfile.Exists(log_dir):
        tf.gfile.MakeDirs(log_dir)
        
    with tf.Graph().as_default() as graph:
        tf.logging.set_verbosity(tf.logging.INFO)

        shape = read_shape_file(MODEL, dataset_dir)

        inputs = tf.placeholder(tf.float32, [None, shape[2]], name='input')
        dropout_rate = tf.placeholder(tf.float32, name='dropout_rate')
        labels = tf.placeholder(tf.int32, name='labels')

        _, probs = model_sp(inputs, num_classes, dropout_rate, is_training=True)
        
        checkpoint_file = tf.train.latest_checkpoint(log_dir)
        # create saver function to restore variables from a checkpoint file
        saver = tf.train.Saver()
        def restore_fn(sess):
            return saver.restore(sess, checkpoint_file)
        clean_dir('../log/test')
        sv = tf.train.Supervisor(logdir='../log/test', summary_op=None, saver=None, init_fn=restore_fn)

        with sv.managed_session() as sess:
            predictions = [['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']]
            for i in tqdm(range(len(dataset))):
                array = [list(dataset.iloc[i, 2:])]
                prob = sess.run(probs, feed_dict={inputs: array, dropout_rate: 0})
                prob = list(prob[0])
                img_name = [dataset.iloc[i, 0]]
                prob = img_name + prob
                predictions.append(prob)
            pd.DataFrame(predictions).to_csv('../submission'+MODEL+'.csv', index=False, header=False)

In [58]:
evaluation_sp('inception_resnet_v2')

INFO:tensorflow:Restoring parameters from ../log/inception_resnet_v2\model.ckpt-280500
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.


100%|██████████| 79726/79726 [03:41<00:00, 360.74it/s]
